In [3]:
import pandas as pd
import numpy as np
import time
from pyecharts.charts import Bar3D


# 1. 读取数据集数据,并查看数据格式

In [4]:
# 1. 定义列表,记录 excel 报名
sheet_names = ['2015',"2016","2017","2018",'会员等级']
# 2. 具体的从数据源文件中读取数据
sheet_datas = pd.read_excel("data/sales.xlsx",sheet_name=sheet_names)
for i in sheet_datas:
    print("="*20)
    print(sheet_datas['2015'].info())
    print(sheet_datas['2015'].describe())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30774 entries, 0 to 30773
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   会员ID    30774 non-null  int64         
 1   订单号     30774 non-null  int64         
 2   提交日期    30774 non-null  datetime64[ns]
 3   订单金额    30774 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 961.8 KB
None
               会员ID           订单号                           提交日期  \
count  3.077400e+04  3.077400e+04                          30774   
mean   2.918779e+10  4.020414e+09  2015-07-01 20:55:49.424839168   
min    2.670000e+02  3.000305e+09            2015-01-01 00:00:00   
25%    1.944122e+10  3.885510e+09            2015-04-02 00:00:00   
50%    3.746545e+10  4.117491e+09            2015-07-02 00:00:00   
75%    3.923593e+10  4.234882e+09            2015-10-01 00:00:00   
max    3.954613e+10  4.282025e+09            2015-12-31 00:00:00   
std    1

# 2. 数据的预处理操作

In [6]:
# 需求: 去重空值,筛选出金额大于1的,新增 max_year_date列,表示该年统计的最后一天
for i in sheet_names[:-1]:
    # 2.2 删除四张表的空值
    sheet_datas[i] = sheet_datas[i].dropna()
    # 2.2 从上述四张表中,筛选出订单金额 >1 的值
    sheet_datas[i] = sheet_datas[i][sheet_datas[i]['订单金额'] > 1]
    # 2.3 新增 max_year_date 列, 表示: 统计每年数据的基本时间: 年中最后一天
    sheet_datas[i].loc[:,"max_year_date"] = sheet_datas[i]['提交日期'].max()
print(sheet_datas['2015'].describe())

               会员ID           订单号                           提交日期  \
count  3.057400e+04  3.057400e+04                          30574   
mean   2.921327e+10  4.020442e+09  2015-07-01 21:10:04.042650624   
min    2.670000e+02  3.000305e+09            2015-01-01 00:00:00   
25%    1.961657e+10  3.885746e+09            2015-04-02 00:00:00   
50%    3.754532e+10  4.117491e+09            2015-07-02 00:00:00   
75%    3.923630e+10  4.234853e+09            2015-10-01 00:00:00   
max    3.954613e+10  4.282025e+09            2015-12-31 00:00:00   
std    1.384598e+10  2.630518e+08                            NaN   

                订单金额                  max_year_date  
count   30574.000000                          30574  
mean      967.270965  2015-12-30 23:59:59.999999744  
min         1.500000            2015-12-31 00:00:00  
25%        59.700000            2015-12-31 00:00:00  
50%       142.000000            2015-12-31 00:00:00  
75%       899.000000            2015-12-31 00:00:00  
max    11

# 2.3 汇总数据

In [7]:
data_merge = pd.concat(list(sheet_datas.values())[:-1])
data_merge

,会员ID,订单号,提交日期,订单金额,max_year_date
0,15278002468,3000304681,2015-01-01,499.0,2015-12-31
1,39236378972,3000305791,2015-01-01,2588.0,2015-12-31
2,38722039578,3000641787,2015-01-01,498.0,2015-12-31
3,11049640063,3000798913,2015-01-01,1572.0,2015-12-31
4,35038752292,3000821546,2015-01-01,10.1,2015-12-31
...,...,...,...,...,...
81344,39229485704,4354225182,2018-12-31,249.0,2018-12-31
81345,39229021075,4354225188,2018-12-31,89.0,2018-12-31
81346,39288976750,4354230034,2018-12-31,48.5,2018-12-31
81347,26772630,4354230163,2018-12-31,3196.0,2018-12-31


In [10]:
# 2.4 给表新增两列数据,date_interval:购买间隔时间 year: 订单所属的年份
data_merge['date_interval'] = data_merge['max_year_date'] - data_merge['提交日期']
# 将上述 date_interval 转成数值
data_merge['date_interval'] = data_merge['date_interval'].dt.days
data_merge

# 给新表增加 year 列,表示该条数数据的所属年份
data_merge.loc[:,"year"] = data_merge['max_year_date'].dt.year
data_merge

# 2.5 按照 年,会员 ID 分组,统计每年每个会员的 最小购买间隔时间,购买总次数,支付的总金额
rfm_gb = data_merge.groupby(['year','会员ID'],as_index=False).agg({
    'date_interval':'min', # 最小购买间隔时间
    '订单号':'count', # 总购买次数
    '订单金额':'sum' # 总支付金额
})
rfm_gb

# 2.6 修改上述别名
rfm_gb.columns = ['year','会员ID','r','f','m']
rfm_gb

,year,会员ID,r,f,m
0,2015,267,197,2,105.0
1,2015,282,251,1,29.7
2,2015,283,340,1,5398.0
3,2015,343,300,1,118.0
4,2015,525,37,3,213.0
...,...,...,...,...,...
148586,2018,39538034299,272,1,49.0
148587,2018,39538034662,189,1,3558.0
148588,2018,39538035729,179,1,3699.0
148589,2018,39545237824,275,1,49.0


# 计算 RFM 各项指标值

In [13]:
# 3.1 划分区间,分别指定 r(Recency:最小购买的间隔时间) f(frequency:购买频率) m(money:购买总金额) 的区间
# 因为我们用的是三分法,即: 3 个区间,所以要指定 4 个值
r_bins = []
f_bins = []
m_bins = []

rfm_gb.iloc[:,2:].describe().T

,count,mean,std,min,25%,50%,75%,max
r,148591.0,165.524043,101.988472,0.0,79.0,156.0,255.0,365.0
f,148591.0,1.365002,2.626953,1.0,1.0,1.0,1.0,130.0
m,148591.0,1323.741329,3753.906883,1.5,69.0,189.0,1199.0,206251.8
